In [4]:
import re
from tqdm import tqdm

In [ ]:
def extract_text_in_square_brackets(text):
    pattern = r'\[(.*?)\]'
    matches = re.findall(pattern, text)
    return matches

def extract_roman_numerals(input_str):
    # Define a regular expression pattern for Roman numerals
    roman_pattern = r'\b[IVXLCDM]+(?=\W|$)'

    # Find all Roman numerals in the input string
    match = re.findall(roman_pattern, input_str)
    matches = []
    for m in match:
        if len(m)>1:
            matches.append(m)
    return matches

def clean_written(written):
    matches = extract_text_in_square_brackets(written)
    if len(matches)!=0:
        print("before clean: ", written)
    for match in matches:
        print(match)
        if match.isdigit() or match.find("Illustration")!=-1:
            written = written.replace(f'[{match}]', "")
            print("replaced")
    
    matches = extract_roman_numerals(written)
    if len(matches)!=0:
        print("before clean: ", written)
    for match in matches:
        print(match)
        # if written.find("CHAPTER")!=-1 and written.find("CHAPTER") == written.find(match)-8 or written.find("STORY")!=-1 and written.find("STORY")==written.find(match)-6:
        #     pass
        # else:
        if written.find(match)==0 or written.find(match)==1:
            written = written.replace(match, "")
            print("replaced")

    written_clean = ""
    for char in written:
        if char.isalnum() or char==" " or char=="'":
            written_clean+=char
            
    return written_clean

In [ ]:
for file in file_names:
    file_path = f"./libriheavy_clean_{file}.txt"
    with open(file_path, "r") as file:
        lines = file.readlines()

    written_clean_list = []
    written_list = []
    spoken_list = []
    edit_distance_list = []
    for line in lines:
        written, spoken = line.split('\t')
        written_clean = clean_written(written)
        written_clean = written_clean.casefold()
        spoken = spoken.casefold()
        edit_distance = char_edit_distance(written_clean, spoken, 2)
        edit_distance_list.append(edit_distance)
        written_list.append(written)
        written_clean_list.append(written_clean)
        spoken_list.append(spoken)
    
    sort_index = [i for i, x in sorted(enumerate(edit_distance_list), key=lambda x: x[1], reverse=True)]

    for idx in sort_index:
        print("idx: ", idx)
        print("edit distance: ", edit_distance_list[idx])
        print("written original: ", written_list[idx])
        print("written: \t", written_clean_list[idx])
        print("spoken: \t", spoken_list[idx])

In [ ]:
def word_edit_distance(str1, str2, deletion_weight=2):
    # Split the input strings into words
    words1 = str1.split()
    words2 = str2.split()

    # Initialize a matrix to store edit distances
    dp = [[0] * (len(words2) + 1) for _ in range(len(words1) + 1)]

    # Initialize the first row and first column of the matrix
    for i in range(len(words1) + 1):
        dp[i][0] = i * deletion_weight
    for j in range(len(words2) + 1):
        dp[0][j] = j

    # Fill in the matrix
    for i in range(1, len(words1) + 1):
        for j in range(1, len(words2) + 1):
            cost = deletion_weight if words1[i - 1] != words2[j - 1] else 0
            dp[i][j] = min(
                dp[i - 1][j] + deletion_weight,  # Deletion
                dp[i][j - 1] + 1,  # Insertion
                dp[i - 1][j - 1] + cost  # Substitution
            )

    # The final value in the matrix represents the word-level edit distance
    return dp[len(words1)][len(words2)]

def char_edit_distance(str1, str2, deletion_weight=2):
    len1, len2 = len(str1), len(str2)

    # Initialize a matrix to store edit distances
    dp = [[0] * (len2 + 1) for _ in range(len1 + 1)]

    # Initialize the first row and first column of the matrix
    for i in range(len1 + 1):
        dp[i][0] = i * deletion_weight
    for j in range(len2 + 1):
        dp[0][j] = j

    # Fill in the matrix
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = deletion_weight if str1[i - 1] != str2[j - 1] else 0
            dp[i][j] = min(
                dp[i - 1][j] + deletion_weight,  # Deletion
                dp[i][j - 1] + 1,  # Insertion
                dp[i - 1][j - 1] + cost  # Substitution
            )

    # The final value in the matrix represents the character-level edit distance
    return dp[len1][len2]